# 🗺️ Mapa de Calor COVID-19 en Europa

Este notebook crea un **mapa de calor interactivo** de Europa mostrando casos de COVID-19 con un slider temporal.

## 1. Importar bibliotecas

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os

## 2. Cargar datos (con caché local)

In [ ]:
# Configuración
url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"
nombre_archivo_local = "owid-covid-data.csv"

# Lógica de caché
if os.path.exists(nombre_archivo_local):
    print(f"✅ Archivo local encontrado. Cargando...")
    df = pd.read_csv(nombre_archivo_local)
else:
    print(f"⚠️ Descargando datos...")
    df = pd.read_csv(url)
    df.to_csv(nombre_archivo_local, index=False)
    print("💾 Guardado localmente.")

df['date'] = pd.to_datetime(df['date'])
print(f"✓ Datos cargados: {len(df):,} filas")

## 3. Filtrar solo países europeos

In [ ]:
# Lista de países europeos
paises_europa = [
    'Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina',
    'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland',
    'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy',
    'Kosovo', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco',
    'Montenegro', 'Netherlands', 'North Macedonia', 'Norway', 'Poland', 'Portugal',
    'Romania', 'Russia', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain',
    'Sweden', 'Switzerland', 'Ukraine', 'United Kingdom', 'Vatican'
]

# Filtrar
df_europa = df[df['location'].isin(paises_europa)].copy()
df_europa = df_europa[df_europa['total_cases'].notna()]

print(f"Rango de fechas: {df_europa['date'].min().strftime('%Y-%m-%d')} a {df_europa['date'].max().strftime('%Y-%m-%d')}")
print(f"Total de registros europeos: {len(df_europa):,}")

## 4. Crear mapa de calor interactivo con slider temporal

In [ ]:
# Crear mapa coroplético
fig = px.choropleth(
    df_europa,
    locations='iso_code',
    color='total_cases',
    hover_name='location',
    hover_data={
        'total_cases': ':,.0f',
        'new_cases': ':,.0f',
        'iso_code': False
    },
    animation_frame=df_europa['date'].dt.strftime('%Y-%m-%d'),
    color_continuous_scale='YlOrRd',
    labels={'total_cases': 'Casos totales'},
    title='📊 Casos de COVID-19 en Europa (Mapa de calor interactivo)',
    scope='europe'
)

# Ajustar diseño
fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    ),
    height=700,
    font=dict(size=14)
)

# Configurar velocidad de animación
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 200
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 100

fig.show()

## 📌 Instrucciones de uso:

1. **Slider inferior**: Arrastra para moverte por las fechas
2. **Botón Play**: Reproduce la animación temporal
3. **Hover**: Pasa el ratón sobre cada país para ver los detalles
4. **Zoom**: Usa los controles superiores para hacer zoom

💡 **Los colores más rojos indican mayor número de casos**

---
## (Opcional) Versión alternativa: usando nuevos casos diarios en vez de totales

In [ ]:
# Versión con nuevos casos (suavizado)
fig2 = px.choropleth(
    df_europa.dropna(subset=['new_cases_smoothed']),
    locations='iso_code',
    color='new_cases_smoothed',
    hover_name='location',
    hover_data={
        'new_cases_smoothed': ':,.0f',
        'total_cases': ':,.0f',
        'iso_code': False
    },
    animation_frame=df_europa['date'].dt.strftime('%Y-%m-%d'),
    color_continuous_scale='Reds',
    labels={'new_cases_smoothed': 'Nuevos casos diarios (promedio 7 días)'},
    title='📊 Nuevos casos diarios COVID-19 en Europa',
    scope='europe',
    range_color=[0, df_europa['new_cases_smoothed'].quantile(0.95)]  # Evitar outliers extremos
)

fig2.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    ),
    height=700
)

fig2.show()